In [1]:
import numpy as np
import matplotlib
import mnist
import scipy
import scipy.sparse.linalg
from scipy.sparse import csr_matrix
from scipy.sparse import identity
import pdb

In [2]:
trainX, trainY = mnist.load_mnist("training", None, './MNIST');
testX, testY = mnist.load_mnist("testing", None, './MNIST');
print("MNIST loaded")

MNIST loaded


In [3]:
#trainX = trainX[0:100, :]
#trainY = trainY[0:100]

In [4]:
class logistic:
    
    
    def __init__(self, eta, lamda, thresh):
        self.eta = eta;
        self.lamda = lamda;
        self.thresh = thresh;
        self.regularized = True;
        
    def load_data(self, string, ori_X, ori_Y):
        """
        if string is "training":
            ori_X, self.Y = mnist.load_mnist("training", None, './MNIST');
        elif string is "testing":
            ori_X, self.Y = mnist.load_mnist("testing", None, './MNIST');
        """
        self.Y = ori_Y;
        s = ori_X.shape;
        self.d = s[1] * s[2] + 1;
        self.n = self.d;
        self.N = s[0];
        self.X = np.reshape(ori_X, (self.N, self.d-1));
        self.X = np.insert(self.X, 0, 1, axis=1);
        self.X_sparse = csr_matrix(self.X, shape=(self.N, self.d))
        print("Loading data " + string + " complete....")
        
    def makeY(self, val):
        temp = np.array(self.Y == val, dtype=float);
        self.Y = np.reshape(temp, (self.N, 1))
        self.Y_sparse = csr_matrix(self.Y, shape=(self.N, 1))
        print("Only looking at class " + str(val) + "!!!")
    
    def initIteration(self):
        self.weights = np.zeros((self.n,1), dtype=float);
        self.iter = 0;
        self.prevLoss = 999999;
            
    def computeProb1(self):
        temp = np.dot(self.X, self.weights);
        temp[temp>100] = 100
        temp = np.exp(temp);
        self.prob1 = temp / (1 + temp)
        self.sampleLoss = self.Y - self.prob1; 
        
    def computeLoss(self):
        #self.loss = np.sqrt(np.sum(self.sampleLoss * self.sampleLoss) / self.N)
        lttemp = np.concatenate([np.float64(self.Y==0), np.float64(self.Y==1)], axis=1);
        rttemp = np.concatenate([1-self.prob1, self.prob1], axis=1)
        self.loss = -np.sum(np.log(np.sum(lttemp * rttemp, axis=1) + 0.0000001)) + self.lamda * np.sum(np.square(self.weights)); 
    
    def updateWeights(self, curIdx, batchSize):
        #print('Weight ' + str(curIdx) + ' : ' + str(self.weights[curIdx]))
        self.batchSize = batchSize;
        idx_from = range(0, self.N, self.batchSize)
        idx_to = idx_from[1:]
        idx_to.extend([self.N])
        temp = self.weights;
        if self.regularized:
            for x in range(0, len(idx_from)):
                self.weights[curIdx] = self.weights[curIdx] + self.eta * np.sum(self.X[idx_from[x]:idx_to[x], curIdx] * self.sampleLoss[idx_from[x]:idx_to[x],0]);
            self.weights[curIdx] = self.weights[curIdx] - self.eta * self.weights[curIdx] * self.lamda;
            #self.weights[curIdx] = self.weights[curIdx] + self.eta * (np.sum(self.X[:, curIdx] * self.sampleLoss[:,0]) - self.weights[curIdx] * self.lamda);
        else:
            for x in range(0, len(idx_from)):
                self.weights[curIdx] = self.weights[curIdx] + self.eta * np.sum(self.X[idx_from[x]:idx_to[x], curIdx] * self.sampleLoss[idx_from[x]:idx_to[x],0]);
            #self.weights[curIdx] = self.weights[curIdx] + self.eta * np.sum(self.X[:, curIdx] * self.sampleLoss[:,0]);
        #print('Weight ' + str(curIdx) + ' : ' + str(self.weights[curIdx]))
    
    def distWeight(self):
        self.weightDist = scipy.linalg.norm(self.weights - temp);
    
    def updateWeightStochastic(self, curIdx, curSmp):
        if self.regularized:
            self.weights[curIdx] = self.weights[curIdx] + self.eta * (self.X[curSmp, curIdx] * self.sampleLoss[curSmp,0] - self.weights[curIdx] * self.lamda);
        else:
            self.weights[curIdx] = self.weights[curIdx] + self.eta * self.X[curSmp, curIdx] * self.sampleLoss[curSmp,0];
    
    def assess(self):
        self.computeProb1();
        self.pred = self.prob1 > 0.5;
        self.correct = np.sum(np.int16(self.pred == (self.Y==1)))
        self.overallAccu = np.float(self.correct) / np.float(self.N);
        self.loss01 = sum(abs(self.pred - self.Y)) / self.N
        print('Accuracy : ' + str(self.overallAccu))
        print('0 / 1 loss : ' + str(self.loss01))
    
    def genGaussFeatWeight(self, n):
        self.n = n;
        self.weight = np.random.randn(self.d, self.n);
    
    def randGaussFeatConv(self):
        self.X = np.dot(self.X, self.weight);

In [5]:
model = logistic(0.1, 0.000001, 0.00001);
model.load_data("training", trainX, trainY);
model.makeY(2);

Loading data training complete....
Only looking at class 2!!!


In [6]:
model.initIteration();
model.computeProb1();
model.computeLoss()
print ('Iteration 0 : Loss value ' + str(model.loss))

while (model.prevLoss - model.loss > model.thresh or model.iter<3):
    model.prevLoss = model.loss;
    model.iter = model.iter + 1;
    for curSmp in range(0, model.N):
        for curIdx in range(0, model.n): 
            model.updateWeightStochastic(curIdx, curSmp)
    model.computeProb1();
    model.computeLoss()
    print ('Iteration ' + str(model.iter) + ' : Loss value ' + str(model.loss))

Iteration 0 : Loss value 41588.8188336
Iteration 1 : Loss value 96033.6512385
Iteration 2 : Loss value 96032.8078497
Iteration 3 : Loss value 96032.3260955
Iteration 4 : Loss value 50781.1611165
Iteration 5 : Loss value 52742.3581499


In [ ]:
model.assess()
model.load_data("testing", testX, testY);
model.makeY(2);
model.assess()